In [ ]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes wandb

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('chizuru_ichinose6.csv')
df.head()

,name,line
0,Chizuru Ichinose,An orange shirt and striped pants…
1,Chizuru Ichinose,"You must be Kazuya-Kun, right?"
2,Chizuru Ichinose,You’re in college? What year?
3,Kazuya Kinoshita,F-First year.
4,Chizuru Ichinose,"What? Me, too!"


In [ ]:
chizuru_index = df[df['name'] == 'Chizuru Ichinose'].index.to_list()
merge_prev = []
for i in chizuru_index:
    if i+1 in chizuru_index:
        merge_prev.append(i+1)
len(merge_prev)

234

In [ ]:
for index, row in df.iloc[::-1].iterrows():
    if index in merge_prev:
        df.at[index - 1, 'line'] += ' ' + row['line']
        df = df.drop(index)

In [ ]:
df = df.reset_index(drop=True)
chizuru_index = df[df['name'] == 'Chizuru Ichinose'].index.to_list()
merge_prev = []
for i in chizuru_index:
    if i+1 in chizuru_index:
        merge_prev.append(i+1)
len(merge_prev)

0

In [ ]:
df = df.iloc[1:]
df = df.reset_index(drop=True)
chizuru_index = df[df['name'] == 'Chizuru Ichinose'].index.to_list()

In [ ]:
df

,name,line
0,Kazuya Kinoshita,F-First year.
1,Chizuru Ichinose,"What? Me, too!"
2,Kazuya Kinoshita,"Oh, but I took a gap year."
3,Chizuru Ichinose,"Ah, you are older than me then. Oops."
4,Kazuya Kinoshita,Damn…
...,...,...
1996,Chizuru Ichinose,It looks like she really does like you. Look ...
1997,Kazuya Kinoshita,Wh-What happened to what you were saying earli...
1998,Chizuru Ichinose,What else are we supposed to do? She’s not li...
1999,Kazuya Kinoshita,"W-Well, yeah…"


In [ ]:
data = pd.DataFrame({'instruction': df.iloc[[x-1 for x in chizuru_index]]['line'].reset_index(drop=True)})
data['output'] = df.iloc[chizuru_index]['line'].reset_index(drop=True)

In [ ]:
data.to_csv('chizuru.csv')

In [ ]:
train_df = data.iloc[:250]
test_df = data.iloc[250:]

In [ ]:
from datasets import load_dataset
from datasets import Dataset

In [ ]:
train_ds = Dataset.from_pandas(train_df, split="train")
test_ds = Dataset.from_pandas(test_df, split="test")

In [ ]:
train_ds

Dataset({
    features: ['instruction', 'output'],
    num_rows: 250
})

In [ ]:
role_play_prompt = "You are Chizuru Ichinose, a rental girlfriend. You project an image of confidence and professionalism while hiding your true feelings. Respond to the following line of dialog in Chizuru's persona."

def chat_template(example):
    example['text'] = f"###Instruction:\n{example['instruction']}\n\n###Response:\n{example['output']}\n\n"
    return example


In [ ]:
train_ds = train_ds.map(chat_template)
test_ds = test_ds.map(chat_template)

In [ ]:
train_ds = train_ds.remove_columns(['instruction', 'output'])
test_ds = test_ds.remove_columns(['instruction', 'output'])

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
base_model = "TinyLlama/TinyLlama-1.1B-step-50K-105b"
new_model = "TinyLlama/chizuru-gpt"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)

model = prepare_model_for_kbit_training(model)

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results",
        num_train_epochs=3,
        per_device_train_batch_size=10,
        gradient_accumulation_steps=1,
        evaluation_strategy="steps",
        eval_steps=1000,
        logging_steps=1,
        optim="paged_adamw_8bit",
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        warmup_steps=10,
        # report_to="wandb",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
trainer.train()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
model.save_pretrained(new_model)